# Daily filtering on MA 200 & Volume

In [1]:
import nselib
from nselib import capital_market

In [2]:
all_listed_stocks = capital_market.bhav_copy_with_delivery('02-05-2025')
stocks = all_listed_stocks[all_listed_stocks['SERIES'] == 'EQ']['SYMBOL'].tolist()

In [3]:
all_listed_stocks.loc[
    all_listed_stocks['SYMBOL'] == 'HDFCBANK', ['SYMBOL', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRICE','TTL_TRD_QNTY']
    ].rename(columns={
        'OPEN_PRICE': 'O',
        'HIGH_PRICE':'H',
        'LOW_PRICE': 'L',
        'CLOSE_PRICE': 'C',
        'TTL_TRD_QNTY': 'Volume'
    })

,SYMBOL,O,H,L,C,Volume
993,HDFCBANK,1929.8,1941.9,1920.0,1925.0,11406024


In [17]:
import concurrent.futures
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

filtered_stocks = []

def process_stock(stock):
    try:
        data = capital_market.price_volume_and_deliverable_position_data(symbol=stock, period='1Y')
    except:
        # skipping; stock with trades less than 1000
        return None

    # -- MA 200 -- 
    MA = 200
    close_series = data[::-1]['ClosePrice'][:MA]
    if close_series.empty:
        return None 
    
    close = close_series.replace({',': ''}, regex=True).astype(float)
    if len(close) < MA:
        return None

    closePrice = close.iloc[0]
    MA_200 = close.mean()

    # TODO: considering adding some buffer
    if closePrice < MA_200:
        # NOTE: skipping; stock has been listed for less than 200 days
        return None

    # -- Volume > 2x 10D Avg --
    try:
        volume_data = capital_market.deliverable_position_data(stock, period='1M')['TradedQty']
        volume = volume_data[::-1].str.replace(',', '', regex=False).astype(int)
    except:
        return None

    current_vol = volume.iloc[0]
    # volume needs to above 100k
    if current_vol > 100_000:
        last_10_days_avg = volume.iloc[1:11].mean()
        if current_vol > last_10_days_avg * 2.25:
            return stock

    return None

print(f'Total no. of stocks: {len(stocks)}')

with concurrent.futures.ThreadPoolExecutor(max_workers=11) as executor:
    futures = [executor.submit(process_stock, stock) for stock in stocks]
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc='Flitering'):
        result = f.result()
        if result:
            filtered_stocks.append(result)

print(f'Filtered stocks: {len(filtered_stocks)}')
filtered_stocks

Total no. of stocks: 2092


Flitering: 100%|██████████| 2092/2092 [04:45<00:00,  7.32it/s]

Filtered stocks: 14


['BSLNIFTY',
 'CREDITACC',
 'ESILVER',
 'FORCEMOT',
 'INDUSTOWER',
 'JETFREIGHT',
 'JMA',
 'JSWSTEEL',
 'NCLIND',
 'PNBHOUSING',
 'PRIVISCL',
 'SSWL',
 'STARCEMENT',
 'ZODIACLOTH']

In [18]:
for stk in filtered_stocks:
    result = all_listed_stocks.loc[
        all_listed_stocks['SYMBOL'] == stk,
        ['SYMBOL', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRICE', 'TTL_TRD_QNTY']
    ]
    
    print(stk)
    print(result.values)



BSLNIFTY
[['BSLNIFTY' 28.4 28.4 27.8 27.98 5127995]]
CREDITACC
[['CREDITACC' 1095.0 1187.9 1095.0 1162.9 4055645]]
ESILVER
[['ESILVER' 96.98 96.98 95.5 95.8 414654]]
FORCEMOT
[['FORCEMOT' 8910.0 10200.0 8892.0 10062.5 810159]]
INDUSTOWER
[['INDUSTOWER' 394.0 395.45 378.0 379.3 19737832]]
JETFREIGHT
[['JETFREIGHT' 16.16 16.24 15.69 16.24 390773]]
JMA
[['JMA' 90.0 99.96 90.0 99.96 158857]]
JSWSTEEL
[['JSWSTEEL' 1021.2 1037.0 948.3 973.2 7915918]]
NCLIND
[['NCLIND' 209.95 221.6 209.95 215.23 255496]]
PNBHOUSING
[['PNBHOUSING' 1071.1 1088.0 1048.3 1052.0 7207887]]
PRIVISCL
[['PRIVISCL' 1971.0 2054.0 1954.4 2049.3 218543]]
SSWL
[['SSWL' 203.3 214.73 201.0 213.49 487198]]
STARCEMENT
[['STARCEMENT' 220.24 224.09 218.05 218.81 899239]]
ZODIACLOTH
[['ZODIACLOTH' 90.13 108.15 90.13 108.15 360151]]
